## 預測價格跟區間

In [2]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/merge_and_cleaned/final_data/short_flight.csv')

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols  = ['出發時段','出發機場代號','抵達時段','航空聯盟','機型分類','假期','是否為平日']
group_col = '抵達機場代號'
num_cols  = ['飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col   = '平均價格_log'
true_col  = '平均價格'
seed      = 123

# --------------------------
# 3. 分艙等並標記真實價格
# --------------------------
eco_raw = df[df['艙等']=='經濟艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()
biz_raw = df[df['艙等']=='商務艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()

# --------------------------
# 4. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    df_enc = pd.get_dummies(df_raw, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 5. 取共同特徵欄位
# --------------------------
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col, group_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 6. 最佳參數（平均價格_log）
# --------------------------
best_params_avg = {
    '經濟艙': {
        'XGBoost': {'n_estimators':250,'learning_rate':0.15,'max_depth':5,'subsample':0.7,'colsample_bytree':0.7}
    },
    '商務艙': {
        'RandomForest': {'n_estimators':250,'max_depth':20,'min_samples_split':9,'min_samples_leaf':3}
    }
}

# --------------------------
# 7. 訓練模型 & 拆分測試集
# --------------------------
def split_and_train(df_enc, cabin_type):
    X = df_enc[common_feats]
    y = df_enc[log_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

    if cabin_type == '經濟艙':
        params = best_params_avg['經濟艙']['XGBoost']
        model = XGBRegressor(tree_method='hist', verbosity=0, random_state=seed, n_jobs=-1, **params)
    else:
        params = best_params_avg['商務艙']['RandomForest']
        model = RandomForestRegressor(random_state=seed, n_jobs=-1, **params)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹[{cabin_type}] 測試集 MSE = {mean_squared_error(y_test, y_pred):.4f}, R2 = {r2_score(y_test, y_pred):.4f}")
    return model, X_train, y_train, X_test

eco_model, eco_X_train, eco_y_train, eco_X_test = split_and_train(eco_enc, '經濟艙')
biz_model, biz_X_train, biz_y_train, biz_X_test = split_and_train(biz_enc, '商務艙')

# --------------------------
# 8. Bootstrap 單次訓練
# --------------------------
def train_and_predict_bootstrap(i, X_train, y_train, X_test, cabin_type):
    # 重新抽樣
    idx = np.random.choice(len(X_train), len(X_train), replace=True)
    Xb, yb = X_train.iloc[idx], y_train.iloc[idx]

    if cabin_type == '經濟艙':
        params = best_params_avg['經濟艙']['XGBoost']
        model = XGBRegressor(tree_method='hist', verbosity=0, random_state=seed+i, n_jobs=-1, **params)
    else:
        params = best_params_avg['商務艙']['RandomForest']
        model = RandomForestRegressor(random_state=seed+i, n_jobs=-1, **params)

    model.fit(Xb, yb)
    return model.predict(X_test)

# --------------------------
# 9. 預測 & 儲存帶區間結果
# --------------------------
def predict_and_save_with_interval(raw_df, model, X_train, y_train, X_test, scaler, cabin_type, prefix, n_boot=100):
    out_dir = f'predict_data/short'
    os.makedirs(out_dir, exist_ok=True)
    idx_test = X_test.index
    test_df = raw_df.loc[idx_test]
    # 誤差方差
    log_pred = model.predict(X_test)
    sigma2 = np.var(log_pred - np.log1p(test_df[true_col]))

    for code in sorted(test_df[group_col].unique()):
        combo = test_df[test_df[group_col]==code][cat_cols+num_cols+[true_col]].drop_duplicates()
        if combo.empty: continue
        enc = pd.get_dummies(combo, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combo[num_cols])
        # 中心預測
        lp = model.predict(enc)
        combo['預測_平均價格'] = np.expm1(lp + 0.5 * sigma2)
        # Bootstrap 區間
        boots = Parallel(n_jobs=-1)(
            delayed(train_and_predict_bootstrap)(i, X_train, y_train, enc, cabin_type)
            for i in range(n_boot)
        )
        arr = np.array(boots)
        combo['預測中位數'] = np.expm1(np.median(arr, axis=0))
        combo['下界'] = np.expm1(np.percentile(arr, 2.5, axis=0))
        combo['上界'] = np.expm1(np.percentile(arr, 97.5, axis=0))
        combo[group_col] = code
        fn = f'{out_dir}/{prefix}_{code}.csv'
        combo.to_csv(fn, index=False)
        print(f"✅[{cabin_type}] 已存：{fn} ({len(combo)}筆)")

predict_and_save_with_interval(eco_raw, eco_model, eco_X_train, eco_y_train, eco_X_test, eco_scaler, '經濟艙', 'eco', n_boot=100)
predict_and_save_with_interval(biz_raw, biz_model, biz_X_train, biz_y_train, biz_X_test, biz_scaler, '商務艙', 'biz', n_boot=100)

🔹[經濟艙] 測試集 MSE = 0.0523, R2 = 0.8599
🔹[商務艙] 測試集 MSE = 0.0083, R2 = 0.9021
✅[經濟艙] 已存：predict_data/short/eco_BKK.csv (298筆)
✅[經濟艙] 已存：predict_data/short/eco_GMP.csv (31筆)
✅[經濟艙] 已存：predict_data/short/eco_HKG.csv (705筆)
✅[經濟艙] 已存：predict_data/short/eco_HND.csv (184筆)
✅[經濟艙] 已存：predict_data/short/eco_ICN.csv (194筆)
✅[經濟艙] 已存：predict_data/short/eco_NRT.csv (384筆)
✅[經濟艙] 已存：predict_data/short/eco_SIN.csv (193筆)
✅[商務艙] 已存：predict_data/short/biz_BKK.csv (181筆)
✅[商務艙] 已存：predict_data/short/biz_GMP.csv (12筆)
✅[商務艙] 已存：predict_data/short/biz_HKG.csv (279筆)
✅[商務艙] 已存：predict_data/short/biz_HND.csv (136筆)
✅[商務艙] 已存：predict_data/short/biz_ICN.csv (146筆)
✅[商務艙] 已存：predict_data/short/biz_NRT.csv (218筆)
✅[商務艙] 已存：predict_data/short/biz_SIN.csv (118筆)
